In [1]:
# !wget https://huggingface.co/datasets/mesolitica/pseudolabel-malaya-speech-stt-train-whisper-large-v3/resolve/main/output-audio.7z.001
# !wget https://huggingface.co/datasets/mesolitica/pseudolabel-malaya-speech-stt-train-whisper-large-v3/resolve/main/output-audio.7z.002
# !wget https://huggingface.co/datasets/mesolitica/pseudolabel-malaya-speech-stt-train-whisper-large-v3/resolve/main/output-audio.7z.003
# !wget https://huggingface.co/datasets/mesolitica/pseudolabel-malaya-speech-stt-train-whisper-large-v3/resolve/main/output-audio.7z.004
# !wget https://huggingface.co/datasets/mesolitica/pseudolabel-malaya-speech-stt-train-whisper-large-v3/resolve/main/output-audio.7z.005
# !wget https://huggingface.co/datasets/mesolitica/pseudolabel-malaya-speech-stt-train-whisper-large-v3/resolve/main/pseudolabel.jsonl

In [2]:
# !pip3 install git+https://github.com/mesolitica/malaya-speech@8fe9cfea37fc32ac63399d9ae5fff22af697f4be

In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [4]:
from transformers.models.bart.modeling_bart import shift_tokens_right
from transformers.models.gpt2 import modeling_gpt2
import malaya
from malaya.text.normalization import cardinal
import malaya_speech
from malaya_speech.utils.subword import merge_sentencepiece_tokens
import re
import itertools
import unicodedata
import json
import numpy as np

tokenizer = malaya.tokenizer.Tokenizer(hypen = False, parliament = False, time = False, time_pukul = False,
                                      temperature = False, distance = False, volume = False, duration = False,
                                      weight = False, date = False, money = False)

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
/home/husein/dev/malaya/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/husein/dev/malaya/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/husein/.local/lib/python3.8/site-packages/whisper/timing.py:58: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-m

In [5]:
def tokenize_and_replace(t):
    tokenized = tokenizer.tokenize(t)
    for i in range(len(tokenized)):
        c = cardinal(tokenized[i])
        if c != tokenized[i]:
            tokenized[i] = c
    return ' '.join(tokenized)

tokenize_and_replace('pada 15 ogos 1940 pihak berikat menyerang perancis selatan serangan ini dipanggil operation dragoon')

'pada lima belas ogos seribu sembilan ratus empat puluh pihak berikat menyerang perancis selatan serangan ini dipanggil operation dragoon'

In [6]:
vocabs = [" ", "a", "e", "n", "i", "t", "o", "u", "s", "k", "r", "l", "h", "d", "m", "g", "y", "b", "p", "w", "c", "f", "j", "v", "z", "0", "1", "x", "2", "q", "5", "3", "4", "6", "9", "8", "7"]
sr = 16000

def preprocessing_text(string):
    
    string = tokenize_and_replace(string)
    string = unicodedata.normalize('NFC', string.lower())
    string = ''.join([c if c in vocabs else ' ' for c in string])
    string = re.sub(r'[ ]+', ' ', string).strip()
    string = (
        ''.join(''.join(s)[:2] for _, s in itertools.groupby(string))
    )
    return string

In [24]:
filtered = []

with open('pseudolabel.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        f = l['audio_filename']
        if not os.path.exists(f):
            continue
        t = l['predict_ms'][41:-13].strip()
        filtered.append((f, t))
        
len(filtered)

1611536

In [10]:
model = malaya_speech.force_alignment.transducer.huggingface(model = 'mesolitica/conformer-medium-mixed')

In [11]:
_ = model.cuda()

In [70]:
!mkdir force-alignment

In [ ]:
from tqdm import tqdm

for i in tqdm(range((len(filtered) // 2) * 0, (len(filtered) // 2) * 1, 1)):
    filename = os.path.join('force-alignment', f'{i}.json')
    if os.path.exists(filename):
        continue
        
    try:
        y, _ = malaya_speech.load(filtered[i][0])
        p = preprocessing_text(filtered[i][1])
        results = model.predict(y, p)
        diag = np.diag(results['alignment']).tolist()

        subwords_alignment = results['subwords_alignment']


        for i in range(len(subwords_alignment)):
            subwords_alignment[i]['start'] = float(subwords_alignment[i]['start'])
            subwords_alignment[i]['end'] = float(subwords_alignment[i]['end'])
            subwords_alignment[i]['score'] = float(subwords_alignment[i]['score'])

        d = {
            'diag': diag,
            'p': p,
            'subwords_alignment': subwords_alignment,
        }

        with open(filename, 'w') as fopen:
            json.dump(d, fopen)
    except:
        with open(filename, 'w') as fopen:
            json.dump(False, fopen)

 49%|██████████████████████████████████████████▏                                           | 394733/805768 [11:13:32<11:56:03,  9.57it/s]